In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')

import re
import os

## .mol headers:

* total atoms
* total bonds
* total angles
* total dihedrals

* coords
* types ∀ atoms
* charges ∀ atoms
* bonds ∀ atoms
* angles ∀ atoms
* dihedrals ∀ atoms

GROMOS FF: 4377 lines

DREIDING FF: 4439 lines

In [2]:
with open("../equilibration_polymer_ij.data", "r") as file:
    file_content = file.readlines()
# Create a DataFrame with each line as a separate row in a single column
data = pd.DataFrame(file_content, columns=['data'])    

In [3]:
coords = data.loc[88:593].copy().reset_index(drop=True)
coords["atom id"] = coords["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
coords["molecule id"] = coords["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
coords["atom type"] = coords["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
coords["charge"] = coords["data"].str.split('\s').apply(lambda x: x[3]).astype(float)
coords["x"] = coords["data"].str.split('\s').apply(lambda x: x[4]).astype(float)
coords["y"] = coords["data"].str.split('\s').apply(lambda x: x[5]).astype(float)
coords["z"] = coords["data"].str.split('\s').apply(lambda x: x[6]).astype(float)
coords = coords.sort_values("atom id").reset_index(drop=True)
coords["element"] = coords["atom type"].map({1:"F",2:"C",3:"O_hyd",
                                             4:"O",5:"S",6:"H"})

In [4]:
coords

,data,atom id,molecule id,atom type,charge,x,y,z,element
0,1 1 1 -0.19227 7.566661627945255 4.10833498450...,1,1,1,-0.19227,7.566662,4.108335,-14.718036,F
1,2 1 1 -0.19227 -15.372510749175992 -4.05817037...,2,1,1,-0.19227,-15.372511,-4.058170,12.049200,F
2,3 1 1 -0.16086 32.28252822770241 9.26827476062...,3,1,1,-0.16086,32.282528,9.268275,-7.796810,F
3,4 1 1 -0.19227 8.28896443979435 11.90582425565...,4,1,1,-0.19227,8.288964,11.905824,2.155806,F
4,5 1 1 -0.19227 -5.903709755545962 8.0839376700...,5,1,1,-0.19227,-5.903710,8.083938,6.594815,F
...,...,...,...,...,...,...,...,...,...
501,502 1 6 0.18144 -15.188887061244928 -18.994274...,502,1,6,0.18144,-15.188887,-18.994275,12.110723,H
502,503 1 6 0.18144 13.75433525855672 13.747924667...,503,1,6,0.18144,13.754335,13.747925,-7.299056,H
503,504 1 6 0.18144 13.596294755505268 9.551620207...,504,1,6,0.18144,13.596295,9.551620,7.464516,H
504,505 1 6 0.18144 -12.956825244068414 8.58110052...,505,1,6,0.18144,-12.956825,8.581101,8.699208,H


In [5]:
bonds_to_be_removed = coords[coords["element"].isin(["H"])]["atom id"].values
bonds_to_be_removed

array([499, 500, 501, 502, 503, 504, 505, 506])

In [6]:
bonds = data.loc[1106:1610].copy().reset_index(drop=True)
bonds["bond id"] = bonds["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
bonds["bond type"] = bonds["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
bonds["atom1"] = bonds["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
bonds["atom2"] = bonds["data"].str.split('\s').apply(lambda x: x[3]).astype(int)

In [9]:
create_ionomer_bonds = bonds[bonds["atom1"].isin(bonds_to_be_removed) | bonds["atom2"].isin(bonds_to_be_removed)]
create_ionomer_bonds.reset_index(drop=True,inplace=True)

In [10]:
create_ionomer_bonds

,data,bond id,bond type,atom1,atom2
0,23 6 489 506\n,23,6,489,506
1,178 6 490 503\n,178,6,490,503
2,202 6 484 499\n,202,6,484,499
3,207 6 488 501\n,207,6,488,501
4,225 6 486 500\n,225,6,486,500
5,354 6 485 505\n,354,6,485,505
6,403 6 487 504\n,403,6,487,504
7,414 6 483 502\n,414,6,483,502


### topotools count bonds from zero :v

In [14]:
print("topo readlammpsdata equilibration_polymer_ij.data full\n")
for i in range(8):
    print('topo delbond {'+f'{create_ionomer_bonds["atom1"][i]-1} {create_ionomer_bonds["atom2"][i]-1}'+'}')

topo readlammpsdata equilibration_polymer_ij.data full

topo delbond {488 505}
topo delbond {489 502}
topo delbond {483 498}
topo delbond {487 500}
topo delbond {485 499}
topo delbond {484 504}
topo delbond {486 503}
topo delbond {482 501}


```
topo getbondlist
{0 305} {1 302} {2 299} {3 298} {4 304} {5 300} {6 301} {7 303} {8 302} {9 298} {10 303} {11 300} {12 305} {13 304} {14 301} {15 299} {16 299} {17 308} {18 310} {19 311} {20 307} {21 306} {22 313} {23 309} {24 312} {25 311} {26 309} {27 306} {28 308} {29 312} {30 313} {31 307} {32 310} {33 314} {34 317} {35 319} {36 321} {37 315} {38 316} {39 318} {40 320} {41 321} {42 315} {43 320} {44 317} {45 314} {46 318} {47 316} {48 319} {49 323} {50 326} {51 327} {52 325} {53 324} {54 328} {55 329} {56 322} {57 323} {58 329} {59 327} {60 322} {61 328} {62 324} {63 325} {64 326} {65 330} {66 332} {67 333} {68 334} {69 336} {70 335} {71 337} {72 331} {73 331} {74 336} {75 334} {76 333} {77 335} {78 337} {79 332} {80 330} {81 341} {82 342} {83 343} {84 344} {85 340} {86 339} {87 338} {88 345} {89 341} {90 342} {91 344} {92 343} {93 340} {94 339} {95 345} {96 338} {97 346} {98 347} {99 348} {100 349} {101 351} {102 352} {103 350} {104 353} {105 350} {106 346} {107 352} {108 347} {109 349} {110 351} {111 353} {112 348} {113 358} {114 356} {115 361} {116 355} {117 357} {118 354} {119 360} {120 359} {121 361} {122 355} {123 358} {124 356} {125 360} {126 354} {127 359} {128 357} {129 363} {130 364} {131 365} {132 367} {133 362} {134 366} {135 368} {136 369} {137 365} {138 363} {139 364} {140 362} {141 368} {142 366} {143 367} {144 369} {145 372} {146 370} {147 371} {148 374} {149 376} {150 377} {151 375} {152 373} {153 370} {154 371} {155 372} {156 374} {157 376} {158 375} {159 377} {160 373} {161 381} {162 378} {163 380} {164 385} {165 384} {166 382} {167 383} {168 379} {169 384} {170 382} {171 383} {172 385} {173 381} {174 378} {175 379} {176 380} {177 386} {178 387} {179 388} {180 389} {181 390} {182 391} {183 393} {184 392} {185 386} {186 387} {187 389} {188 388} {189 390} {190 391} {191 392} {192 393} {193 394} {194 395} {195 396} {196 397} {197 399} {198 400} {199 401} {200 398} {201 402} {202 403} {203 405} {204 408} {205 409} {206 402} {207 403} {208 404} {209 405} {210 407} {211 408} {212 406} {213 409} {214 402} {215 404} {216 407} {217 406} {218 410} {219 411} {220 413} {221 412} {222 415} {223 416} {224 414} {225 417} {226 410} {227 411} {228 412} {229 413} {230 414} {231 415} {232 416} {233 417} {234 418} {235 419} {236 421} {237 420} {238 424} {239 422} {240 423} {241 425} {242 426} {243 427} {244 429} {245 428} {246 430} {247 431} {248 433} {249 432} {250 426} {251 427} {252 429} {253 428} {254 430} {255 431} {256 432} {257 433} {258 426} {259 427} {260 429} {261 428} {262 432} {263 430} {264 431} {265 433} {266 434} {267 435} {268 437} {269 436} {270 440} {271 438} {272 439} {273 441} {274 434} {275 435} {276 436} {277 437} {278 440} {279 438} {280 439} {281 441} {282 442} {283 443} {284 445} {285 444} {286 447} {287 448} {288 446} {289 449} {290 442} {291 443} {292 445} {293 444} {294 447} {295 448} {296 449} {297 446} {298 407} {298 309} {299 307} {300 409} {300 311} {301 403} {301 312} {302 404} {302 310} {303 408} {303 308} {304 405} {304 313} {305 406} {305 306} {306 318} {307 319} {308 314} {309 321} {310 317} {311 316} {312 315} {313 320} {314 325} {315 324} {316 329} {317 328} {318 323} {319 327} {320 326} {321 322} {322 336} {323 331} {324 335} {325 333} {326 334} {327 332} {328 337} {329 330} {330 345} {331 339} {332 338} {333 340} {334 343} {335 341} {336 344} {337 342} {338 350} {339 352} {340 351} {341 346} {342 347} {343 348} {344 349} {345 353} {346 354} {347 360} {348 356} {349 357} {350 361} {351 359} {352 358} {353 355} {354 368} {355 364} {356 366} {357 369} {358 365} {359 362} {360 367} {361 363} {362 374} {363 370} {364 371} {365 372} {366 375} {367 376} {368 377} {369 373} {370 379} {371 380} {372 378} {373 385} {374 381} {375 383} {376 382} {377 384} {378 391} {379 393} {380 392} {381 390} {382 387} {383 388} {384 386} {385 389} {386 394} {387 395} {388 396} {389 397} {390 399} {391 400} {392 398} {393 401} {394 402} {394 450} {395 403} {395 451} {396 404} {396 452} {397 405} {397 453} {398 406} {398 457} {399 407} {399 454} {400 408} {400 455} {401 409} {401 456} {410 418} {410 450} {411 419} {411 451} {412 421} {412 452} {413 420} {413 453} {414 424} {414 457} {415 422} {415 454} {416 423} {416 455} {417 425} {417 456} {418 426} {418 458} {419 427} {419 459} {420 428} {420 460} {421 429} {421 461} {422 430} {422 463} {423 431} {423 464} {424 432} {424 465} {425 433} {425 462} {434 442} {434 458} {435 443} {435 459} {436 445} {436 461} {437 444} {437 460} {438 447} {438 463} {439 448} {439 464} {440 446} {440 462} {441 449} {441 465} {442 490} {443 491} {444 493} {445 492} {446 496} {447 494} {448 495} {449 497} {466 490} {467 491} {468 492} {469 493} {470 497} {471 494} {472 495} {473 496} {474 490} {475 491} {476 492} {477 493} {478 494} {479 495} {480 497} {481 496} {482 501} {482 490} {483 498} {483 491} {484 504} {484 492} {485 499} {485 493} {486 503} {486 494} {487 500} {487 495} {488 505} {488 497} {489 502} {489 496}
```

In [ ]:
angles = data.loc[1614:2597].copy().reset_index(drop=True)
angles["angle id"] = angles["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
angles["angle type"] = angles["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
angles["atom1"] = angles["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
angles["atom2"] = angles["data"].str.split('\s').apply(lambda x: x[3]).astype(int)
angles["atom3"] = angles["data"].str.split('\s').apply(lambda x: x[4]).astype(int)

In [ ]:
dihedrals = data.loc[2601:4008].copy().reset_index(drop=True)
dihedrals["dihedral id"] = dihedrals["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
dihedrals["dihedral type"] = dihedrals["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
dihedrals["atom1"] = dihedrals["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
dihedrals["atom2"] = dihedrals["data"].str.split('\s').apply(lambda x: x[3]).astype(int)
dihedrals["atom3"] = dihedrals["data"].str.split('\s').apply(lambda x: x[4]).astype(int)
dihedrals["atom4"] = dihedrals["data"].str.split('\s').apply(lambda x: x[5]).astype(int)

In [ ]:
def write_mol_file():
    f = open("nafion_dreiding.mol", "w")
    f.write("# Created by Karina Chiñas Fuentes | Python | 07.08.24\n\n")
    f.write("\n506 atoms\n505 bonds\n984 angles\n1407 dihedrals")
    f.write("\n\n")
    f.write("Coords\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['x'][i]}\t{coords['y'][i]}\t{coords['z'][i]}\n")
    f.write("\nTypes\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['atom type'][i]}\n")
    f.write("\nCharges\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['charge'][i]}\n")
    f.write("\nBonds\n\n")
    for i in range(len(bonds)):
        f.write(f"{bonds['bond id'][i]}\t{bonds['bond type'][i]}\t"+
                f"{bonds['atom1'][i]}\t{bonds['atom2'][i]}\n")
    f.write("\nAngles\n\n")
    for i in range(len(angles)):
        f.write(f"{angles['angle id'][i]}\t{angles['angle type'][i]}\t"+
                f"{angles['atom1'][i]}\t{angles['atom2'][i]}\t{angles['atom3'][i]}\n")
    f.write("\nDihedrals\n\n")
    for i in range(len(dihedrals)):
        f.write(f"{dihedrals['dihedral id'][i]}\t{dihedrals['dihedral type'][i]}\t{dihedrals['atom1'][i]}\t"+
                f"{dihedrals['atom2'][i]}\t{dihedrals['atom3'][i]}\t{dihedrals['atom4'][i]}\n")

In [ ]:
#write_mol_file()

## parm.lammps headers:

* mass ∀ types
* pair_coeff ∀ types (individual and crossed)
* bond_coeff ∀ types
* angle_coeff ∀ types
* dihedral_coeff ∀ types

(Did it by hand)